<a href="https://colab.research.google.com/github/imenFerjani/Fish-School-Search-CNN/blob/main/FishSchoolSeach_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
################# Import Packages ###########################

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import string
import keras
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
plt.style.use('seaborn')
from copy import deepcopy,copy
import pandas as pd
import time
from tqdm import tqdm


import random
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, Embedding, SpatialDropout1D, Conv1D, MaxPool1D, Flatten, concatenate, Dense, Dropout

from keras.preprocessing import sequence
from keras.datasets import imdb

from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix



import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier


In [2]:
# WORD-level classification
MAX_NUM_WORDS     = 5000
EMBEDDING_DIM     = 300
MAX_SEQ_LENGTH    = 512

# GENERAL
DROPOUT_RATE      = 0.5
#NB_CLASSES        = 1

# LEARNING
BATCH_SIZE        = 32
NB_EPOCHS         = 1
#RUNS              = 5


In [3]:
# loading imdb database
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_NUM_WORDS)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=MAX_SEQ_LENGTH)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_SEQ_LENGTH)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=MAX_SEQ_LENGTH)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_SEQ_LENGTH)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
17473536/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 512)
x_test shape: (25000, 512)
Pad sequences (samples x time)
x_train shape: (25000, 512)
x_test shape: (25000, 512)


In [4]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(25000, 512) (25000, 512) (25000,) (25000,)


In [20]:
def add_convpool_block(layers, max_out_ch, max_kernel_size):
  '''This function returns a convolution layer with random filters number and random size each'''
  filters_lenghts = random.choice([3,4])
  deb = random.choice([2,3,4,5,6,7])
  conv_kernel = random.choice([[i for i in range(deb, deb+filters_lenghts)], [deb for i in range(filters_lenghts)]])
    
  out_channel = random.choice([64,128,512])
  #np.random.randint(128, max_out_ch)
  #conv_kernel = np.random.randint(3, max_kernel_size)

  layers.append({"type": "convpool_block", "ou_c": out_channel, "kernel": conv_kernel})

  return layers

def add_fc(layers, max_fc_neurons):
  '''This function returns a fully connected layer '''
  layers.append({"type": "fc", "ou_c": np.random.randint(100, max_fc_neurons), "kernel": -1})
    
  return layers

def model_compile(list_layers,nb_classes,drop_rate,max_num_words, embedding_dim,max_seq_length):
  '''this function compiles the generated model'''
  input_text = Input(shape=(max_seq_length,))
  embedding_layer = Embedding(max_num_words, embedding_dim, input_length=max_seq_length)(input_text)
  text_embed = SpatialDropout1D(0.2)(embedding_layer)
  filter_lengths = list_layers[0]['kernel']
  conv_layers = []
  
  for filter_length in filter_lengths:
    conv_layer = Conv1D(filters= list_layers[0]['ou_c'], kernel_size=filter_length, padding='valid',
                        strides=1, activation='relu')(text_embed)
    maxpooling = MaxPool1D(pool_size=max_seq_length - filter_length + 1)(conv_layer)
    flatten = Flatten()(maxpooling)
    conv_layers.append(flatten)
  sentence_embed = concatenate(inputs=conv_layers)
  sentence_embed = Dropout(0.5)(sentence_embed)
  dense_layer = Dense(list_layers[-1]['ou_c'], activation='relu')(sentence_embed)
  output = Dense(nb_classes, activation='sigmoid')(dense_layer)
  #output = Dense(nb_classes, activation='softmax')(dense_layer)
  model_c = Model(input_text, output)
  model_c.compile(loss='binary_crossentropy', metrics=['acc'], optimizer= 'adam')
  #model_c.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer= 'adam')
  return model_c

def model_fit(model_t,x_train, y_train, batch_size,nb_epochs, full_training = False):
  '''This function fits the model with total or partial data depending on full_training option'''
  if full_training==False:
    #IMDB
    x_train1, y_train1 = x_train[:1000], y_train[:1000]
    x_test1, y_test1 = x_test[:1000], y_test[:1000]
    #x_train = x_train[:round(x_train.shape[0]/4)]
    #y_train = y_train[:round(y_train.shape[0]/4)]
    
    hist = model_t.fit(x=x_train1, y=y_train1, batch_size=batch_size, epochs=nb_epochs)
       
    return model_t.evaluate(x_test1, y_test1, verbose=True)
  else:
    model_t.summary()
    hist = model_t.fit(x=x_train, y=y_train,batch_size=batch_size, epochs=nb_epochs)
    return model_t.evaluate(x_test, y_test, verbose=True)

  

In [ ]:
import random
layers=[]
filters_lenghts = random.choice([3,4])
deb = random.choice([2,3,4,5,6,7])
conv_kernel = random.choice([[i for i in range(deb, deb+filters_lenghts)], [deb for i in range(filters_lenghts)]])
layers.append({"type": "convpool_block", "ou_c": 22, "kernel": conv_kernel})
layers[0]['kernel']

[5, 5, 5]

In [19]:
# Defining the class fish 
class Fish:
  def __init__(self,max_num_words,embedding_dim,max_seq_length,max_kernel_size,nb_classes,
               min_depth, max_depth,conv_pool_prob, fc_prob, max_out_ch,max_fc_neurons,n_iter):
    # Fish Attributes
    self.delta_pos = np.nan
    self.delta_cost = 0.0
    self.weight = n_iter / 4.0
    #self.cost = np.nan
    self.has_improved = False

    # Model attributes
    self.depth = np.random.randint(min_depth, max_depth)
    self.max_num_words = max_num_words
    self.embedding_dim = embedding_dim
    self.max_seq_length = max_seq_length
    self.max_kernel_size = max_kernel_size
    self.nb_classes = nb_classes
    
    self.conv_pool_prob = conv_pool_prob
    self.fc_prob = fc_prob
    self.max_fc_neurons = max_fc_neurons
    self.max_out_ch = max_out_ch
        
    self.layers = []
    self.nb_convpool_blocks=None
    self.nb_fc_layers=None
    self.acc = None
    self.loss= None
    self.model=None

    self.initialization()
  
  def __str__(self):
    string = ""
    for z in range(len(self.layers)):
      
      if self.layers[z]["type"]== "convpool_block":
        string = string + "Conv_layer" + " || " + "MaxPoling_layer" + " || "
      else:
          string = string +  "FullyConnected_layer" + " || "

    return '\033[94m'+'\033[1m'+string+'\033[0m'

    # Build fish (CNN) architecture
  def update_layers(self):
    nb_convpool_blocks= len([i for i in range(len(list_layers)) if self.layers[i]["type"] == "convpool_block"])
    nb_fc_layers= len([i for i in range(len(list_layers)) if self.layers[i]["type"] == "fc"])
    return nb_convpool_blocks,nb_fc_layers
    
  def initialization(self):
    
    self.nb_convpool_blocks = 1
    #round((self.depth * self.conv_pool_prob)/2)
    print("cnvpool=",self.nb_convpool_blocks)
    self.nb_fc_layers = 1
    #self.depth - self.nb_convpool_blocks * 2
    print("fc=",self.nb_fc_layers)
    # First block is always a convolution layer followed by a pooling layer
    for i in range(self.nb_convpool_blocks):
      self.layers = add_convpool_block(self.layers, self.max_out_ch, self.max_kernel_size)
      
    for i in range(self.nb_fc_layers):
      self.layers = add_fc(self.layers, self.max_fc_neurons)
      
    # Last layer is always a fully connected layer    
    #self.layers[-1] = {"type": "fc", "ou_c": self.nb_classes, "kernel": -1}
    
    print(self.layers)
    self.model= model_compile(self.layers,self.nb_classes,drop_rate,self.max_num_words, self.embedding_dim, self.max_seq_length)
       
    self.loss,self.acc= model_fit(self.model,x_train, y_train, batch_size,nb_epochs)
       

##### Model methods ####
  

  def model_delete(self):
    # This is used to free up memory during FSS training
    del self.model
    keras.backend.clear_session()
    tf.reset_default_graph()
    self.model = None

##### FSS methods ####

  def move_to(self,nb_convblocks_new, nb_fc_new):
    change=False
    print("Steps:",nb_convblocks_new, nb_fc_new)
    list_layers1=deepcopy(self.layers)
    if nb_convblocks_new > 0:
      change=True
      filters_lenghts = random.choice([3,4])
      deb = random.choice([2,3,4,5,6,7])
      conv_kernel = random.choice([[i for i in range(deb, deb+filters_lenghts)], [deb for i in range(filters_lenghts)]])
    
      out_channel = random.choice([64,128,512])
      #print("add convpool")
      for i in range(nb_convblocks_new):        
        list_layers1.insert(0,{"type": "convpool_block", "ou_c": out_channel, "kernel": conv_kernel})
    elif nb_convblocks_new < 0 and abs(nb_convblocks_new) < self.nb_convpool_blocks:
      #print("del convpool")
      change=True 
      for i in range(int(abs(nb_convblocks_new))):
        del list_layers1[0]
    elif nb_convblocks_new < 0 and self.nb_convpool_blocks > 1:
      change=True
      del list_layers1[:self.nb_convpool_blocks-1]
    else:
      pass;      
    if nb_fc_new > 0:
      #print("add fc")
      change=True
      for i in range(nb_fc_new):
        list_layers1.insert(-1,{"type": "fc", "ou_c": np.random.randint(1, self.max_fc_neurons), "kernel": -1})
    elif nb_fc_new < 0 and abs(nb_fc_new) < self.nb_fc_layers :
      #print("del fc")
      change=True
      print("architecture before modification:",list_layers1)
      for i in range(int(abs(nb_fc_new))):
        del list_layers1[-2]
    elif nb_fc_new < 0 and self.nb_fc_layers > 1 :
      change=True
      del list_layers1[len(list_layers1)-self.nb_fc_layers:len(list_layers1)-1]
    else:
      pass;
    return list_layers1,change

  def individual_movement(self):
    print("Starting individual movement")
    #list_layers = self.layers
    try_times=1
    max_acc = self.acc
    print("initial fish accuracy:", max_acc)
    for i in range(try_times):
      print("try number:",i)
      step_x = np.random.randint(-2,3)
      
      step_y = np.random.randint(-2,3)
      if (step_x,step_y != 0,0):
        list_layers,change = self.move_to(step_x,step_y)
        print("compiling model")
        model_ind = model_compile(list_layers,self.nb_classes, drop_rate,
                              self.max_num_words, self.embedding_dim, self.max_seq_length)
        print("training model")
        loss,acc = model_fit(model_ind,x_train, y_train, batch_size,nb_epochs)
        print("Fish accuracy after modification",acc)
        if acc > max_acc:
          print("Improvement after modification:"+'\033[92m' +"YES"+ '\033[0m')
          max_acc=acc
          self.delta_cost = acc - self.acc
          self.acc=acc
        #nb_convpool_blocks,nb_fc_layers = self.update_layers()
          self.delta_pos= (step_x ,step_y)
          self.layers= list_layers
          
          self.nb_convpool_blocks = len([x for x in list_layers if x["type"]=="convpool_block"]) 
          self.nb_fc_layers = len([x for x in list_layers if x["type"]=="fc"])
          self.loss,self.acc = loss,acc
          self.model=model_ind
          self.has_improved=True
        else: print("Improvement after modification:"+'\033[91m' +"NO"+ '\033[0m')        
    return self
        
  
  def collective_movement(self,nb_convpool_blocks,nb_fc_layers):
    list_layers,change = self.move_to(nb_convpool_blocks,nb_fc_layers)
    if change:
      self.model = model_compile(list_layers,self.nb_classes, drop_rate, self.max_num_words, self.embedding_dim, self.max_seq_length)
      loss,acc = model_fit(self.model,x_train, y_train, batch_size,nb_epochs)
      #ecrire dans le fichier
      self.delta_cost= acc - self.acc
      if self.delta_cost > 0:
        print("Improvement after collective movement:"+'\033[92m' +"YES"+ '\033[0m')
        self.has_improved= True
        self.delta_pos= (nb_convpool_blocks ,nb_fc_layers)
        self.layers= list_layers
        self.nb_convpool_blocks+=nb_convpool_blocks
        self.nb_fc_layers += nb_fc_layers
        self.loss,self.acc = loss,acc
      else:
        print("Improvement after collective movement:"+'\033[91m' +"NO"+ '\033[0m')
        self.has_improved=False

    return self


In [18]:
class School:
  def __init__(self,n_iter,school_size):
    self.n_iter = n_iter
    self.dim=2
    self.school_size = school_size
    self.prev_weight_school = 0.0
    self.curr_weight_school = 0.0
    self.best_fish = None
    self.best_fish_acc=0.0
    self.best_fish_loss=0.0
    self.best_fish_model=None

    

    self.optimum_cost_tracking_iter = []
    self.optimum_cost_tracking_eval = []

    self.school = []
    for i in range(school_size):
      fish = Fish(max_num_words,embedding_dim,max_seq_length,max_kernel_size,nb_classes,min_depth,
                  max_depth,conv_pool_prob, fc_prob, max_out_ch,max_fc_neurons,n_iter)
      self.school.append(fish)
      
      self.curr_weight_school += fish.weight
      

    self.prev_weight_school = self.curr_weight_school
    
    self.update_best_fish()
    
     
  def update_best_fish(self):
    for fish in self.school:
      if self.best_fish_acc < fish.acc:
        self.best_fish = copy(fish)
        self.best_fish_acc = fish.acc
        self.best_fish_loss = fish.loss
        self.best_fish_model=fish.model
    
  def max_delta_cost(self):
    max_ = 0
    for fish in self.school:
      if max_ < fish.delta_cost:
        max_ = fish.delta_cost
    return max_

  def total_school_weight(self):
    self.prev_weight_school = self.curr_weight_school
    self.curr_weight_school = 0.0
    for fish in self.school:
      self.curr_weight_school += fish.acc

  def calculate_barycenter(self):
    barycenter = np.zeros((self.dim,) ,dtype=np.int) # 2 : 1 convpool blocks ; 1 fc layers
    density = 0.0
    for fish in self.school:
      density += fish.acc
      barycenter[0] += (fish.nb_convpool_blocks * fish.acc)
      barycenter[1] += (fish.nb_fc_layers * fish.acc)
    for dim in range(self.dim):
      barycenter[dim] = barycenter[dim] / density

    return barycenter

  def update_steps(self, curr_iter):
    self.curr_step_individual = self.step_individual_init - curr_iter * float(
    self.step_individual_init - self.step_individual_final) / self.n_iter
    self.curr_step_volitive = self.step_volitive_init - curr_iter * float(
    self.step_volitive_init - self.step_volitive_final) / self.n_iter
  
  
  def feeding(self):
    max_delta_cost = self.max_delta_cost()
    for fish in self.school:
      if max_delta_cost :
        fish.weight += (fish.delta_cost / max_delta_cost)
            
  def collective_movement(self):
    def sortThird(val): 
      return val[2] 
    self.total_school_weight()
    list1 =  [(fish.nb_convpool_blocks,fish.nb_fc_layers,fish.acc) for fish in self.school ]
    list1.sort(key = sortThird, reverse = True)    
    averge_convpool_blocks = np.mean([list1[i][0] for i in range (len(list1)//2)])
    print("Average convpool",averge_convpool_blocks) 
    average_fc_layers = np.mean([list1[i][1] for i in range (len(list1)//2)]) 
    print("Average fc",average_fc_layers) 
    for fish in self.school:
      #ecrire dans le fichier
      if self.curr_weight_school > self.prev_weight_school:
        if fish.nb_convpool_blocks > averge_convpool_blocks:
          nb_convpool_blocks = -1 * abs((averge_convpool_blocks - fish.nb_convpool_blocks)) * 0.5 
        else:
          nb_convpool_blocks = abs((averge_convpool_blocks - fish.nb_convpool_blocks)) * 0.5 
        if fish.nb_fc_layers > average_fc_layers:
          nb_fc_layers =  -1 * abs((average_fc_layers - fish.nb_fc_layers)) * 0.5 
        else:
          nb_fc_layers =  abs((average_fc_layers - fish.nb_fc_layers)) * 0.5 
      else:
        if fish.nb_convpool_blocks > averge_convpool_blocks:
          nb_convpool_blocks = abs((averge_convpool_blocks - fish.nb_convpool_blocks)) * 0.5 
        else:
          nb_convpool_blocks = -1* abs((averge_convpool_blocks - fish.nb_convpool_blocks)) * 0.5 
        if fish.nb_fc_layers > average_fc_layers:
          nb_fc_layers =  abs((average_fc_layers - fish.nb_fc_layers)) * 0.5 
        else:
          nb_fc_layers = -1 * abs((average_fc_layers - fish.nb_fc_layers)) * 0.5 
      nb_convpool_blocks,nb_fc_layers = int(nb_convpool_blocks),int(nb_fc_layers)
      if (nb_convpool_blocks,nb_fc_layers)!=(0,0):
        print('\033[95m'+"collective movement steps:"+'\033[0m',int(nb_convpool_blocks),int(nb_fc_layers))
        fish.collective_movement(int(nb_convpool_blocks),int(nb_fc_layers))
      else:
        print('\033[91m'+"null collective movement"+'\033[0m')
  
  def optimize(self):
    #self.__init__(max_num_words,embedding_dim,max_seq_length,max_kernel_size,nb_classes,
               #min_depth, max_depth,conv_pool_prob, fc_prob, max_out_ch,max_fc_neurons,n_iter,school_size)
    track_Best_fish=[]
    track_fish_before_indiv_mvt=[]
    track_fish_after_indiv_mvt=[]
    track_fish_before_collec_mvt=[]
    track_fish_before_collec_mvt=[]
    
    for i in tqdm(range(self.n_iter)):
      find.write("Iteration number: " + str(i)+'\n')
      fcoll.write("Iteration number: " + str(i)+'\n')
      fiter.write("Iteration number: " + str(i)+'\n')
      k=1
      for fish in self.school:
        find.write("fish number "+str(k)+'before individual movement :'+str(fish.acc)+'---')
        fish.individual_movement()
        find.write('after individual movement :'+str(fish.acc)+'\n')
        k+=1
      fiter.write('Best fish before individual movement:'+ str(self.best_fish_acc)+'----')
      self.update_best_fish()
      fiter.write('Best fish after individual movement:'+ str(self.best_fish_acc)+'\n')
      #self.feeding()
      fcoll.write('Best fish before collective movement:'+ str(self.best_fish_acc)+'----')
      self.collective_movement()
      #self.collective_volitive_movement()
      #self.update_steps(i)
      self.update_best_fish()
      fcoll.write('Best fish after collective movement:'+ str(self.best_fish_acc)+'\n')
      
      print ("Iteration: ", i, " Cost: ", self.best_fish.acc)
    

In [11]:
def create_dir(path):
    directory = os.path.dirname(path)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

In [17]:
max_num_words= MAX_NUM_WORDS
embedding_dim=EMBEDDING_DIM
max_seq_length=MAX_SEQ_LENGTH
max_kernel_size=7
nb_classes=1

min_depth=2
max_depth=10
# Probability of each layer type (should sum to 1)
conv_pool_prob=0.7
fc_prob=0.3

max_out_ch= 256
max_fc_neurons= 300

drop_rate=0.5
batch_size=BATCH_SIZE
nb_epochs = NB_EPOCHS
epochs_full_training = 1

runs_time=[]
# FSS parameters
results_path = "Executions/"
num_runs = 1
school_size = 10
num_iterations = 5
all_gbest_par=[]
create_dir(results_path)
find=open(results_path+'/individual_movement.txt', 'w')
find.write("individual movement track\n")
fcoll=open(results_path+'/collective_movement.txt', 'w')
fcoll.write("collective movement track\n")
fbest=open(results_path+'/Best_fish.txt', 'w')
fbest.write("global best fish track\n")
fiter=open(results_path+'/best_fish_OverIterations.txt', 'w')
fiter.write("Best fish Over Iterations track\n")
      
for i in range (num_runs):
  print("******************Run number: " + str(i)+'************************\n')
  fbest.write("******************Run number: " + str(i)+'************************\n')
  find.write("******************Run number: " + str(i)+'************************\n')
  fcoll.write("******************Run number: " + str(i)+'************************\n')
  fiter.write("******************Run number: " + str(i)+'************************\n')
  start_time = time.time()
  s = School(num_iterations,school_size)
  s.optimize()
  fbest.write("Best fish accuracy: " + str(s.best_fish_acc)+'\n')
  print(s.best_fish_acc)
  # Plot current Best_fish
  matplotlib.use('Agg')
  plt.plot(s.best_fish_acc)
  plt.xlabel("Run")
  plt.ylabel("Fish Best accuracy")
  plt.savefig(results_path + "gBest-iter-" + str(i) + ".png")
  plt.close()

  print('Best architecture found: ')
  print(s.best_fish)
  fbest.write("Best fish architecture found: " + s.best_fish.__str__() +'\n')
  
  #fbest.write("Parameters of Best fish architecture found: " + s.best_fish.layer'\n')  
  np.save(results_path + "Best_inter_" + str(i) + "_acc_history.npy", s.best_fish_acc)

  #np.save(results_path + "gBest_iter_" + str(i) + "_test_acc_history.npy", s.gBest_test_acc)

  end_time = time.time()

  running_time = end_time - start_time
  fbest.write("Running Time: " + str(running_time) +'\n')
  runs_time.append(running_time)

  # Fully train and evaluate the Best model found
   
  n_parameters = model_fit(s.best_fish.model,x_train, y_train, batch_size,epochs_full_training, full_training = True)
  #n_parameters = model_fit(s.best_fish.model,x_train, y_train, batch_size,epochs_full_training, full_training = True)
 
  all_gbest_par.append(n_parameters)
  print(all_gbest_par)
  fbest.write("final model evaluation: " + '\n')
  l1=map(lambda x:str(x)+';', all_gbest_par)
  fbest.writelines(l1)
  para=[]
  for nn in s.best_fish.layers:
    para=para+list(nn.values())
  l1=map(lambda x:str(x)+';', para)
  fbest.writelines(l1)
  # Evaluate the fully trained gBest model
  result=s.best_fish.model.evaluate(x_test,  y_test, verbose=True)
find.close()
fcoll.close()
fiter.close()
fbest.close()
        




******************Run number: 0************************

cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 64, 'kernel': [3, 4, 5, 6]}, {'type': 'fc', 'ou_c': 122, 'kernel': -1}]
ok1
ok2
32/32 [==============================] - 0s 8ms/step - loss: 0.6862 - acc: 0.6490
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 512, 'kernel': [2, 3, 4, 5]}, {'type': 'fc', 'ou_c': 101, 'kernel': -1}]
ok1
ok2
32/32 [==============================] - 1s 26ms/step - loss: 0.6795 - acc: 0.5460
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 64, 'kernel': [7, 7, 7]}, {'type': 'fc', 'ou_c': 202, 'kernel': -1}]
ok1
ok2
32/32 [==============================] - 0s 7ms/step - loss: 0.6876 - acc: 0.6040
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 64, 'kernel': [4, 4, 4, 4]}, {'type': 'fc', 'ou_c': 178, 'kernel': -1}]
ok1
ok2
32/32 [==============================] - 0s 9ms/step - loss: 0.6961 - acc: 0.4740
ok3
cnvpool= 1
fc= 1
[{'type': 'convpool_block', 'ou_c': 512, 'kernel': 

  0%|          | 0/5 [00:00<?, ?it/s]

Starting individual movement
initial fish accuracy: 0.6489999890327454
try number: 0
Steps: 1 -2
compiling model
training model
32/32 [==============================] - 0s 8ms/step - loss: 0.6920 - acc: 0.4770
Fish accuracy after modification 0.47699999809265137
Improvement after modification:NO
Starting individual movement
initial fish accuracy: 0.5460000038146973
try number: 0
Steps: 2 -1
compiling model
training model
32/32 [==============================] - 0s 7ms/step - loss: 0.6843 - acc: 0.5260
Fish accuracy after modification 0.5260000228881836
Improvement after modification:NO
Starting individual movement
initial fish accuracy: 0.6039999723434448
try number: 0
Steps: 2 0
compiling model
training model
32/32 [==============================] - 0s 6ms/step - loss: 0.6986 - acc: 0.4740
Fish accuracy after modification 0.4740000069141388
Improvement after modification:NO
Starting individual movement
initial fish accuracy: 0.4740000069141388
try number: 0
Steps: 0 -2
compiling model

 20%|██        | 1/5 [00:41<02:47, 41.98s/it]

Improvement after collective movement:NO
null collective movement
null collective movement
null collective movement
null collective movement
null collective movement
Iteration:  0  Cost:  0.6489999890327454
Starting individual movement
initial fish accuracy: 0.6489999890327454
try number: 0
Steps: 0 -1
compiling model
training model
32/32 [==============================] - 0s 8ms/step - loss: 0.6889 - acc: 0.5010
Fish accuracy after modification 0.5009999871253967
Improvement after modification:NO
Starting individual movement
initial fish accuracy: 0.5460000038146973
try number: 0
Steps: -1 2
compiling model
training model
32/32 [==============================] - 1s 28ms/step - loss: 0.6887 - acc: 0.4770
Fish accuracy after modification 0.47699999809265137
Improvement after modification:NO
Starting individual movement
initial fish accuracy: 0.6039999723434448
try number: 0
Steps: -1 -2
compiling model
training model
32/32 [==============================] - 0s 8ms/step - loss: 0.6868 - 

...

 80%|████████  | 4/5 [02:54<00:43, 43.88s/it]

Improvement after collective movement:NO
null collective movement
null collective movement
null collective movement
null collective movement
Iteration:  3  Cost:  0.6930000185966492
Starting individual movement
initial fish accuracy: 0.6489999890327454
try number: 0
Steps: 1 2
compiling model
training model
32/32 [==============================] - 0s 7ms/step - loss: 0.6892 - acc: 0.5560
Fish accuracy after modification 0.5559999942779541
Improvement after modification:NO
Starting individual movement
initial fish accuracy: 0.5460000038146973
try number: 0
Steps: 1 -2
compiling model
training model
32/32 [==============================] - 0s 9ms/step - loss: 0.6842 - acc: 0.6140
Fish accuracy after modification 0.6140000224113464
Improvement after modification:YES
Starting individual movement
initial fish accuracy: 0.6759999990463257
try number: 0
Steps: 2 2
compiling model
training model
32/32 [==============================] - 1s 12ms/step - loss: 0.6855 - acc: 0.5260
Fish accuracy af

100%|██████████| 5/5 [03:40<00:00, 44.00s/it]

Improvement after collective movement:NO
null collective movement
null collective movement
null collective movement
null collective movement
Iteration:  4  Cost:  0.6930000185966492
0.6930000185966492
Best architecture found: 
Conv_layer || MaxPoling_layer || FullyConnected_layer || 
Model: "model_155"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_156 (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 embedding_155 (Embedding)      (None, 512, 300)     1500000     ['input_156[0][0]']              
                                                                                                  
 spatial_dropout1d_155 (Spatial  (None, 512, 300)    0           ['embedding_155[0][0]']          
 Dr

782/782 [==============================] - 6s 7ms/step - loss: 0.2784 - acc: 0.8828
[[0.27844780683517456, 0.8828399777412415]]
